### Calibration / sanity check for the simualtion

### Pulsed-NMR simulation


In [1]:
import os
import sys

print(os.path.abspath(os.curdir))
os.chdir("..")  # go to parent folder
os.chdir("..")  # go to parent folder

print(os.path.abspath(os.curdir))
sys.path.insert(0, os.path.abspath(os.curdir))
from SimuTools import Sample, MagField, Simulation, TTL

# from SimuTools import *
# from DataAnalysis import *
from functioncache import check
import numpy as np
# sqdsensor = SQUID(name="Virtual SQUID", Mf=1.0, Rf=1.0)  # in Ohm

c:\Users\zhenf\D\Yu0702\NMR-simulator\Tests\20250326-Simulations-for-calibration
c:\Users\zhenf\D\Yu0702\NMR-simulator


In [2]:
ExampleSample10MHzT = Sample(
    name="TestSample",  # name of the atom/molecule
    gyroratio=2
    * np.pi
    * (10)
    * 10**6,  # [Hz/T]. Remember input it like 2 * np.pi * 11.777*10**6
    numofnuclei=1,  #
    tempunit="K",  # temperature scale
    boilpt=165.051,  # [K]
    meltpt=161.40,  # [K]
    density_liquid=2.942,  # [g/cm^3] at boiling point
    density_gas=5.894 * 10**3,  # [g/cm^3] at STP
    density_solid=None,  # [g/cm^3]
    molarmass=131.2930,  # [g/mol]
    spindenisty_liquid=None,  # [mol/cm^3]
    spindenisty_gas=None,  # [g/cm^3] at STP
    spindenisty_solid=None,  # [mol/cm^3]
    shareofpeaks=[1.0],  # array or list.
    T2=None,  # [s]
    T1=1000,  # [s]
    pol=1,
    verbose=False,
)

In [3]:
# biasField = MagField(name="bias field") # excitation field in the rotating frame

In [4]:
excField = MagField(name="excitation field") # excitation field in the rotating frame
# excField.nu = 1e6  # [Hz]

In [5]:
simu = Simulation(
                name="TestSample 10MHzT",
                sample=ExampleSample10MHzT,  # class Sample
                # gyroratio=(2*np.pi)*11.777*10**6,  # [Hz/T]
                init_time=0.0,  # [s]
                station=None,
                init_magamp=1.0,
                init_magtheta=0.0,  # [rad]
                init_magphi=0.0,  # [rad]
                B0z=(1e6 - 40)
                / (ExampleSample10MHzT.gyroratio / (2 * np.pi)),  # [T]
                simuRate=(6696.42871094),  # max(10000, a
                excField=excField,
                verbose=False,
            )

TypeError: bad operand type for abs(): 'NoneType'

In [ ]:
listofGammaandSAmp = []
listofT2andavgMtsq = []
listofspectrum = []
gaNNorder_arr = np.arange(-2.0, -11.0, -0.5)
ALPnuorder_arr = np.arange(3.0, 7.0, 1)
for ALPnuorder in [6]:
    check(ALPnuorder)
    for T2star in [
        0.2
    ]:  # 0.0001, 0.001, , 0.1 5.0, 10.0, , 20.0, 30.0, 40.0, 50.0, 100.0 0.1,  [2,10,50,100], 10**(1.5), 100, 10**(2.5), 1000
        # [0.1, 1, 10**(0.5), 10, 10**(1.5), 100, 10**(2.5), 1000]
        #
        # check(T2star)
        for gaNNorder in [-1.4]:
            # check(gaNNorder)
            # axionwind = AxionWind(
            # 	name='ALP',
            # 	nu=10 ** ALPnuorder,  # compton frequency in [Hz]
            # 	Gamma=5. * 10. ** (-6),  # spectrum linewidth.
            # 	gaNN=10**gaNNorder,  # in Gev^-1
            # 	direction_solar=np.array([1, 90*np.pi/180., - 90. * np.pi / 180]),
            # 	verbose=False
            # 	)
            simu = Simulation(
                name="TestSample 10MHzT",
                sample=ExampleSample10MHzT,  # class Sample
                # gyroratio=(2*np.pi)*11.777*10**6,  # [Hz/T]
                init_time=0.0,  # [s]
                station=Mainz,
                init_magamp=1.0,
                init_magtheta=0.0,  # [rad]
                init_magphi=0.0,  # [rad]
                B0z=(10**ALPnuorder - 40)
                / (ExampleSample10MHzT.gyroratio / (2 * np.pi)),  # [T]
                simuRate=(6696.42871094),  # max(10000, a
                excField=excField,
                verbose=False,
            )

            # set excitation field
            simuDur = 1
            simuTimeStamp = np.linspace(0, simuDur, num=int(simuDur / simu.simuRate))
            pulseDur = 100e-6
            B1 = (np.pi /2) / (simu.sample.gyroratio * pulseDur)
            duty_func = partial(TTL, start=0, stop=pulseDur)
            excField.setSinPulseExcitation(
                timeStamp=simuTimeStamp,
                B1=B1,  # amplitude of the excitation pulse in [T]
                nu_e=0.1,  # Hz
                init_phase=0,
                duty_func=duty_func,
                verbose= False,
            )
            # check(magnetization.nu_rot)
            # # tic = time.perf_counter()
            # magnetization.GenerateParam(
            # 		numofcohT=1,  #max(10 * T2star, 1)
            # 		method='RandomJump',  #'ThermalLight' 'RandomJump' 'InfCoherence'
            # 		showplt=False,  # whether to plot B_ALP
            # 		plotrate=0.1,
            # 		verbose=False
            # 		)
            # toc = time.perf_counter()
            # print(f'GenerateParam time consumption = {toc-tic:.3f} s')
            # simurate 1000, numofcohT=100/axionwind.cohT, not usejit. time = 2.3 s
            # simurate 1000, numofcohT=100/axionwind.cohT, usejit @jit. time = 1.8 s
            # simurate 1000, numofcohT=100/axionwind.cohT, usejit @jit(types blabla). time = 0.1 s
            # simurate 1000, numofcohT=1000/axionwind.cohT, usejit @jit(types blabla). time = 1.1 s
            # simurate 1000, numofcohT=100000/axionwind.cohT, usejit @jit(types blabla). time = 107.1 s

            tic = time.perf_counter()
            simu.GenerateTrajectory(verbose=False)
            toc = time.perf_counter()
            # print(f'GenerateTrajectory time consumption = {toc-tic:.3f} s')
            # simurate 1000, numofcohT=100/axionwind.cohT, not usejit. time = 4.2 s
            # simurate 1000, numofcohT=100/axionwind.cohT, usejit @jit. time = 1.6 s
            # simurate 1000, numofcohT=100/axionwind.cohT, usejit @jit(types blabla). time = 0.208 s
            # simurate 1000, numofcohT=1000/axionwind.cohT, usejit @jit(types blabla). time = 2.069 s

            simu.MonitorTrajectory(plotrate=10**3, verbose=True)
            simu.VisualizeTrajectory3D(
                plotrate=10**3,  # [Hz]
                # rotframe=True,
                verbose=False,
            )
            # magnetization.SaveTrajectory(
            # 		h5fpathandname=f'K:/CASPEr data/20220522_NMRKineticSimu_data_test2/sample_IDEN/'+\
            # 						f'simudata_test2_ALPwind.nu_1e{ALPnuorder:.1f}_gaNN_1e{gaNNorder:.1f}_samplelinewidth_{samplelinewidth:g}_'+\
            # 							f'T1_{magnetization.T1:g}_T2_{magnetization.T2:.3g}',
            # 		saveintv=1,  # int
            # 		verbose=False
            # )
            print(
                f"***************************************************************************"
            )
            print(f"T2* = {simu.T2:e}")
            simu.StatTrajectory(verbose=True)
            print(
                f"**************************************************************************"
            )
            # processdata = False
            # if processdata: